In [15]:
import torch
from IPython.core.debugger import set_trace

def my_relu(x: torch.FloatTensor) -> torch.FloatTensor:
    # set_trace() # <-- this!
    if x < 0: 
        print("if branch. x:", x)
        return x * 0
    return x
x = torch.tensor(-2., requires_grad=True)
res = my_relu(x)
res.backward()

print("val: ", res.item())
print("grad", x.grad.item())

if branch. x: tensor(-2., requires_grad=True)
val:  -0.0
grad 0.0


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models

#from ipdb import set_trace # <-- for debugging

DEVICE = torch.device(
    "cuda:0" if torch.cuda.is_available()
    else "cpu"
)

In [22]:
a = torch.rand(3,3)
a.shape

torch.Size([3, 3])

In [27]:
a + a @ a.t();

In [29]:
a = a.to(DEVICE)
a + a @ a.t();

In [31]:
a.cpu(); # a.cuda()

In [41]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5),
    nn.ReLU(),
    nn.Conv2d(20, 64, 5),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
)

In [42]:
batch = torch.rand(16,1,24,24)
model(batch); # forward pass

In [45]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)
        self.conv2 = nn.Conv2d(20,64,5)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return F.adaptive_avg_pool2d(x,1)

In [54]:
import collections
 
try:
    from collections import OrderedDict
except ImportError:
    OrderedDict = dict

def bn_conv_relu(in_channels, out_channels, kernel_size=5):
    return nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.Conv2d(in_channels, out_channels, kernel_size),
        nn.ReLU(),
    )

model = nn.Sequential(
    OrderedDict({
        "conv1": bn_conv_relu(1,20,7),
        "conv2": bn_conv_relu(20,64),
        "aap": nn.AdaptiveAvgPool2d(1),
    })
)

In [56]:
model

Sequential(
  (conv1): Sequential(
    (0): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(1, 20, kernel_size=(7, 7), stride=(1, 1))
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU()
  )
  (aap): AdaptiveAvgPool2d(output_size=1)
)

In [62]:
model = model.to(DEVICE);

In [67]:
lin = nn.Linear(1,3,bias=True)
print(lin)

Linear(in_features=1, out_features=3, bias=True)


In [69]:
lin.state_dict()

OrderedDict([('weight',
              tensor([[0.5995],
                      [0.1643],
                      [0.1482]])),
             ('bias', tensor([-0.9066, -0.9253, -0.1082]))])

In [71]:
for p in lin.parameters():
    print(p)
    print()

Parameter containing:
tensor([[0.5995],
        [0.1643],
        [0.1482]], requires_grad=True)

Parameter containing:
tensor([-0.9066, -0.9253, -0.1082], requires_grad=True)



In [76]:
from torch.utils.data import Dataset, DataLoader

def init_weights(module):
    if isinstance(module, nn.Linear):
        print(f"Initializing {module} with uniform")
        nn.init.uniform_(module.weight)
        
lin.apply(init_weights);

Initializing Linear(in_features=1, out_features=3, bias=True) with uniform


In [81]:
dataset = torchvision.datasets.MNIST("data/raw/", download=True)

In [82]:
len(dataset)

60000

In [84]:
dataset[1]

(<PIL.Image.Image image mode=L size=28x28 at 0x7FA1786E0C10>, 0)

In [87]:
print(f"Label: {dataset[1][1]}")
dataset[1][0]

Label: 0


In [92]:
dataloader = DataLoader(
    dataset,
    batch_size=64,
    num_workers=4,
    shuffle=True,
)

In [96]:
# ML is model + loss + optimizer
model # from above
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [100]:
optimizer = optim.SGD(
    [s
        {"params": model.conv1.parameters(), "lr": 0.001},
        {"params": model.conv2.parameters()}
    ],
    lr=0.01,
)

In [104]:
def fit(model, loss_fn, optimizer, train_dl, valid_dl, n_epochs: int):
    for epoch in range(n_epochs):
        # TRAIN
        model.train()
        for x, y in train_dl:
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            y_ = model(x)
            loss = loss_fn(y, y_)
            loss.backward()
            optimizer.step()
            
        # EVAL
        model.eval()
        with torch.no_grad():
            for x, y in valid_dl:
                x, y = x.to(DEVICE), y,to(DEVICE)
                y_ = model(x)
                loss = loss_fn(y, y_)

In [106]:
model = nn.DataParallel(model) 

In [109]:
torchvision.models.resnet18(pretrained=True)
torchvision.models.densenet121(pretrained=True)
torchvision.models.inception_v3(pretrained=True)
torchvision.models.squeezenet1_1(pretrained=True);